In [14]:
from openai import OpenAI 
import yaml
import os
from dotenv import load_dotenv

load_dotenv()

MODEL="gpt-4o"
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as an env var>")
DB_INDEX_FILE_LOCAL_PATH = os.environ.get("DB_INDEX_FILE_LOCAL_PATH", "<path to the index.json file>")
DB_ROOT_LOCAL_PATH = os.environ.get("DB_ROOT_LOCAL_PATH", "<path to the root directory of the database>")

openAiClient = OpenAI(api_key=OPENAI_API_KEY)

with open("../prompts/memoryClassifier.yml", "r") as file:
    group_classifier = yaml.safe_load(file)

system_prompt_for_classifier = group_classifier["prompt"]

with open("../prompts/memoryAssistant.yml", "r") as file:
    group_classifier = yaml.safe_load(file)

system_prompt_for_assistant = group_classifier["prompt"]

with open(DB_INDEX_FILE_LOCAL_PATH, "r") as file:
    metadata = file.read()

In [15]:
def create_messages_for_classifier(query, metadata):
    return [
        {
            "role": "system",
            "content": system_prompt_for_classifier
        },
        {
            "role": "user",
            "content": f"Query: ${query}"
        },
        {
            "role": "user",
            "content": f"Metadata: ${metadata}"
        }        
    ]

def create_messages_for_assistant(query, memories):
    return [
        {
            "role": "system",
            "content": system_prompt_for_assistant
        },
        {
            "role": "user",
            "content": f"Query: ${query}"
        },
        {
            "role": "user",
            "content": f"Memories: ${memories}"
        }
    ]

In [19]:
query = "any movie to recommend?"
completion = openAiClient.chat.completions.create(
    model=MODEL,
    temperature=0,
    stream=False,
    response_format={"type": "json_object"},
    messages=create_messages_for_classifier(query, metadata)),

completion_content = completion[0].choices[0].message.content
print(completion_content)

{
  "thinking_process": "The user's query is about movie recommendations. Based on the provided metadata, the most relevant category is 'AREAS' under 'ENTERTAINMENT/tv_shows', which includes a list of TV shows the user has watched. Although the query is specifically about movies, the TV shows list might provide some context or related recommendations. No other categories or files seem directly relevant to movie recommendations.",
  "keys": [
    "AREAS/ENTERTAINMENT/tv_shows"
  ]
}


In [20]:
# parse the response in JSON and print the value of *data*
import json


keys = json.loads(completion_content)["keys"]

memories = []
for key in keys:
    memory_asset_path = os.path.join(DB_ROOT_LOCAL_PATH, key)
    with open(memory_asset_path, "r") as file:
        memory_asset_content = file.read()
        memories.append({
            "path": key,
            "content": memory_asset_content
        })

memories_string = json.dumps(memories)

completion = openAiClient.chat.completions.create(
    model=MODEL,
    temperature=0,
    stream=False,
    response_format={"type": "json_object"},
    messages=create_messages_for_assistant(query, memories_string)),

completion_content = completion[0].choices[0].message.content
print(completion_content)    


{
  "thinking_process": "The user has asked for a movie recommendation, but the provided memories only contain a list of TV shows they have watched. To provide a relevant movie recommendation, I need to consider the genres and themes of the TV shows the user enjoys. Based on the variety of shows listed, the user seems to enjoy a mix of drama, thriller, sci-fi, comedy, and fantasy. I will recommend a movie that aligns with these interests.",
  "response": "Based on your interest in a variety of TV shows including drama, thriller, sci-fi, comedy, and fantasy, I recommend watching 'Inception'. It's a mind-bending thriller with a complex plot and stunning visuals, directed by Christopher Nolan. If you're in the mood for something lighter, 'The Grand Budapest Hotel' by Wes Anderson is a fantastic comedy-drama with a unique style and great performances."
}
